# Social Graphs 2021 - Assignment 2 

In [ ]:
# Styles and data. If this cell block is not already hidden/minimized, feel free to do so, as it only contains the styles for the plots, 
# and the characters.csv in written form, as it allows for reruns of the code, if needed.

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import wordcloud
import nltk
import networkx as nx

import re
import urllib
import requests

import os